In [1]:
import torch
from glob import glob
import pickle

In [2]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained('../checkpoints_16enclayers_agg/', 'checkpoint_best.pt', '/home/beckmann/kf_bin')
roberta.cuda()
assert isinstance(roberta.model, torch.nn.Module)

In [3]:
def path_to_indseq(table, path):
    indseqs = []
    videos = sorted(glob('{}/*'.format(path)))
    for video in videos:
        frames = sorted(glob(video + '/*'))
        ind_seq = [table[frame.split('/')[-1]] for frame in frames]
        indseqs.append(ind_seq)
    return indseqs

In [4]:
def load_bpe(path, splits=['train','test','valid']):
    all_content=[]
    for split in splits:
        with open(path + split + '.bpe') as f:
            content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.strip() for x in content]
        print(len(content))
        all_content.extend(content)
    return all_content

In [5]:
with open('/home/beckmann/kf_bpe/imnet_table.pkl', 'rb') as f:
    rawdata = f.read()
imnet_table = pickle.loads(rawdata)

path = '/mnt/storage02/tamedia/keyframes/nn_keyframes/'
videos = sorted(glob('{}/*'.format(path)))

bpe_data = path_to_indseq(imnet_table, path)

In [ ]:
roberta_features = []
for bpe_d in bpe_data:
    data_string = ' '.join(str(ind) for ind in bpe_d)
    tokens = roberta.encode(data_string)
    roberta_features.append(roberta.extract_features(tokens)[0][0].cpu().data.numpy())

512